In [ ]:
!pip install typing_extensions==4.7.1 --upgrade

In [ ]:
!pip install peft datasets transformers bitsandbytes

In [ ]:
!python -m pip install --upgrade pip
# 내부적으로 사용하는 툴의 오류를 해결하기 위해

In [ ]:
!pip install langchain

In [ ]:
!pip install langchain-community

In [ ]:
!pip install chromadb

In [ ]:
!pip install langchain_openai

In [ ]:
!pip install --upgrade transformers peft

In [ ]:
!pip install --upgrade torch transformers peft


In [ ]:
!pip uninstall numpy -y
!pip install "numpy<2"
!pip install faiss-gpu

In [ ]:
!pip install tf-keras

In [86]:
# 기본 성능 체크

In [ ]:
import os
import torch
import transformers
from datasets import load_from_disk
from transformers import (
    BitsAndBytesConfig,
    AutoModelForCausalLM,
    AutoTokenizer,
    Trainer,
    TextStreamer,
    pipeline,
    DataCollatorForSeq2Seq,
    TrainingArguments
)
from peft import (
    LoraConfig,
    prepare_model_for_kbit_training,
    get_peft_model,
    get_peft_model_state_dict,
    set_peft_model_state_dict,
    TaskType,
    PeftModel
)
from trl import SFTTrainer

In [ ]:
BASE_MODEL = "yanolja/EEVE-Korean-10.8B-v1.0"

model = AutoModelForCausalLM.from_pretrained(BASE_MODEL, load_in_4bit=True, device_map="auto")
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL)

In [ ]:
prompt = "자율주행자동차에 대해 알려줘"

# 텍스트 생성을 위한 파이프라인 설정
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=256) # max_new_tokens: 생성할 최대 토큰 수
outputs = pipe(
    prompt,
    do_sample=True, # 샘플링 전략 사용. 확률 분포를 기반으로 다음 토큰을 선택
    temperature=0.2, # 샘플링의 다양성을 조절하는 파라미터. 값이 높을수록 랜덤성 증가
    top_k=50, # 다음 토큰을 선택할 때 상위 k개의 후보 토큰 중에서 선택. 여기에서는 상위 50개의 후보 토큰 중에서 샘플링
    top_p=0.95, # 누적 확률이 p가 될 때까지 후보 토큰을 포함
    repetition_penalty=1.2, # 반복 패널티를 적용하여 같은 단어나 구절이 반복되는 것 방지
    add_special_tokens=True # 모델이 입력 프롬프트의 시작과 끝을 명확히 인식할 수 있도록 특별 토큰 추가
)
print(outputs[0]["generated_text"][len(prompt):]) # 입력 프롬프트 이후에 생성된 텍스트만 출력

In [ ]:
# qlora

In [16]:
import os
import torch
import transformers
import pandas as pd
from datasets import load_dataset, Dataset, concatenate_datasets
from transformers import AutoModelForCausalLM, AutoTokenizer
from langchain.schema import Document
import json


In [37]:
import torch
from transformers import pipeline, AutoModelForCausalLM, AutoTokenizer, TrainingArguments, Trainer, DataCollatorForSeq2Seq
# AutroModelForCausalLM: GPT 기반의 생성형 모델 로드하는 클래스 / DataCollatorForSeq2Seq : 데이터 배치, 데이터 로더와 함꼐 사용, 함수 제공공
from datasets import load_dataset
from peft import LoraConfig, get_peft_model, PeftModel
#PeftModel: 모델 로드드
import bitsandbytes as bnb
# bnb: 양자화에 도움을 줌/8bit의 연산을 지원해주는 라이브러리/모델을 좀 더 최적화하는데 도움
import torch.nn.functional as F
# 신경망에 적용하는 함수들

In [14]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [70]:
file_paths = {
    "term": "./term.json"
}
def load_json(path):
    with open(path, 'r', encoding='utf-8') as f:
        return json.load(f)
        
def convert_list_to_documents(data_list, doc_type):
    return [
        Document(page_content=json.dumps(item, ensure_ascii=False), metadata={"type": doc_type})
        for item in data_list
    ]
dataset_origin= convert_list_to_documents(load_json(file_paths["term"]), "term")

In [18]:
from typing_extensions import TypeIs

##### 모델 설정

In [22]:
model_name = "yanolja/EEVE-Korean-10.8B-v1.0"

In [76]:
# 4-bit 양자화된 모델 로드를 위한 설정
bnb_config={
    'load_in_4bit':True,                        # 4비트 양자화 적용할 것인지
    'bnb_4bit_compute_dtype':torch.float16,     # 4비트의 연산을 수행할 때 어떤 데이터 타입을 쓸 것인지-torch.float16: 속도 최적화
    'bnb_4bit_quant_type':'nf4',                # 양자화 방식에 대한 type: nf4: 4-bit의 normal float(교안 참고): 성능 개선을 위해: 정규화화
    'device_map':'auto'                         # GPU가 여러 대일때 모두 사용할 수 있게끔 자동 설정
}

In [23]:
# 토크나이저 및 모델 로드 (모델 로드 시 4-bit 양자화 설정)
tokenizer = AutoTokenizer.from_pretrained(model_name)
base_model = AutoModelForCausalLM.from_pretrained(model_name, **bnb_config)

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

In [30]:
dataset = load_dataset("json", data_files="./term.json")


Generating train split: 0 examples [00:00, ? examples/s]

In [25]:
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

In [31]:
dataset

DatasetDict({
    train: Dataset({
        features: ['term', 'desc'],
        num_rows: 44
    })
})

In [53]:
def preprocess_data_conversation(examples):
    input_texts = []
    target_texts = []
    
    for desc, term in zip(examples['desc'], examples['term']):
        # 대화형 프롬프트
        input_text = f"질문: {desc}에 해당하는 용어는?\n답변:"
        target_text = f" {term}"
        
        input_texts.append(input_text)
        target_texts.append(target_text)
    
    # 입력+출력을 하나로 결합 (GPT 스타일)
    full_texts = [inp + tgt for inp, tgt in zip(input_texts, target_texts)]
    
    tokenized = tokenizer(
        full_texts,
        max_length=512,
        truncation=True,
        padding=True,
        return_tensors=None
    )
    
    # 자기지도학습용: input_ids를 labels로도 사용
    tokenized["labels"] = tokenized["input_ids"].copy()
    
    return tokenized

##### 파인 튜닝을 위한 LoRA 설정

In [38]:
# 파인튜닝을 위한 LoRA 설정
lora_config = LoraConfig(
    r=32,
    lora_alpha=64,
    lora_dropout=0.1,
    bias='none',# 가중치 행렬만, 편향은 없음
    task_type='CAUSAL_LM'# lora가 적용될 대상 모델 타입과 맞추어준다.
)

In [39]:
#LoRA 적용

model = get_peft_model(base_model, lora_config)
model.enable_input_require_grads()          # 모델에 대해 입력값으로 받은 값에 대해 gradient를 사용하게끔하는 설정
model.gradient_checkpointing_enable()       # 체크포인트로 중간 저장: 모델이 커지게 되면 중간에 수행해준 내용들을 저장하는것이 효율적
model.print_trainable_parameters()          # 학습가능한 파라미터의 수를 출력

trainable params: 20,447,232 || all params: 10,825,371,648 || trainable%: 0.1889


##### Training 설정

In [77]:
# 설정
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)  # 동적 데이터를 저장하면서 자동으로 다음 데이터를 불러옴: model의 타입에 맞추어 tokenizer을 사용함/

training_args = TrainingArguments(
    output_dir = './q_lora_korqa',      # 
    eval_strategy='no',              # 검증: 하지 않음
    save_strategy='steps',              #
    per_device_train_batch_size=4,      # 배치 사이즈: 트레인
    per_device_eval_batch_size=4,       # 배치 사이즈: eval
    gradient_accumulation_steps=8,      # 가중치를 누산하여 한번에 계산 : 8번만큼의 가중치를 누적해 놓았다가 한번에 계산
    learning_rate=2e-4,                 # 학습률
    weight_decay=0.01,                  # L2 정규화 적용 비율
    num_train_epochs=10,                 # 몇번의 epoch을 진행할 것인지
    logging_dir='./logs',               # log 남길 dir
    logging_steps=100,                  # log 남길 빈도수
    save_total_limit=2,                 # 체크포인트 최대 개수: 가장 최근의 2개 저장
    fp16=True,                          # 
    push_to_hub=False,                  # hub: hugginface에서의 허브 - true: 자동으로 hub에 저장
    report_to='none',                    # 학습결과를 표현할 수 있는 툴에 전달달
    remove_unused_columns=False
)

In [78]:
train_dataset = dataset['train'].map(preprocess_data_conversation, batched=True, remove_columns=dataset['train'].column_names)

# Trainer 설정
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,  # 전처리된 데이터셋 사용
    tokenizer=tokenizer,
    data_collator=data_collator
)

Map:   0%|          | 0/44 [00:00<?, ? examples/s]

/tmp/ipykernel_1094/1096101301.py:4: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [79]:
# 모델 학습
trainer.train()

Step,Training Loss


TrainOutput(global_step=20, training_loss=0.1571187734603882, metrics={'train_runtime': 216.9381, 'train_samples_per_second': 2.028, 'train_steps_per_second': 0.092, 'total_flos': 1.330835763265536e+16, 'train_loss': 0.1571187734603882, 'epoch': 10.0})

In [80]:
# 모델 저장
trainer.save_model("workspace/q_lora_korqa/checkpoint-15")

In [81]:
from transformers import AutoConfig

trained_model_path='./q_lora_korqa/checkpoint-15'

config = AutoConfig.from_pretrained(model_name) # 원본 모델에 대한 설정 로드
config.save_pretrained(trained_model_path)      # 체크포인트 경로에 설정 저장: 재사용하기 위해해

In [82]:
# 학습시킨 adaptor 이어붙이기
adapter_model_path = './q_lora_korqa/checkpoint-15'

base_model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype='auto', device_map='auto')

model = PeftModel.from_pretrained(base_model, adapter_model_path)



Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

Some parameters are on the meta device because they were offloaded to the cpu.


ValueError: Can't find 'adapter_config.json' at './q_lora_korqa/checkpoint-15'

In [ ]:
OPENAI_API_KEY= 

##### API 제한 -> 순서대로 넣기

In [71]:
from langchain.vectorstores import Chroma  # persist 지원
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

# 1. 청크 크기 조정 (500~1000 권장)
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50,
    length_function=len,
    is_separator_regex=False,
)

# 2. 문서 분할
all_splits = text_splitter.split_documents(dataset_origin)

embedding_model = OpenAIEmbeddings(model='text-embedding-3-large', openai_api_key=OPENAI_API_KEY)

# 4. Chroma DB에 배치 처리로 저장
batch_size = 100  # 한 번에 처리할 청크 수
vectorstore = Chroma.from_documents(
    documents=all_splits[:batch_size],  # 첫 배치
    embedding=embedding_model,
    persist_directory="./vectordb1"
)

# 남은 청크를 순차적으로 추가
for i in range(batch_size, len(all_splits), batch_size):
    batch = all_splits[i:i+batch_size]
    vectorstore.add_documents(
        documents=batch,
        embedding=embedding_model
    )

vectorstore.persist()  



/tmp/ipykernel_1094/153068654.py:34: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vectorstore.persist()


In [87]:
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from langchain_openai import ChatOpenAI
from transformers import pipeline
from langchain.llms import HuggingFacePipeline


# 2. 프롬프트 템플릿
prompt = PromptTemplate(
    template="""<|instruction|>
주어진 문서에서 질문과 가장 관련성이 높은 정보를 우선적으로 활용하여 답변하세요.
문서에 없는 내용은 추측하지 말고, 문서 기반으로만 답변하세요.

<|query|>
사용자 질문: {question}

<|retrieved_context|>
{context}

<|response_format|>
1. 직접 답변 (한 문장)
2. 상세 설명 (2-3문장)  
3. 법적 근거 (있는 경우)
4. 추가 참고사항

<|system|>
다음 예시들을 참고하여 동일한 형식과 품질로 답변하세요.

<|example1|>
질문: 자율주행시스템이란?
답변: "「자율주행자동차 상용화 촉진 및 지원에 관한 법률」 제2조 제1항 제2호에 따른 자율주행시스템을 말한다. 이 경우 그 종류는 완전 자율주행시스템, 부분 자율주행시스템 등 행정안전부령으로 정하는 바에 따라 세분할 수 있다."

<|example2|>  
질문: 보행자우선도로이란?
답변: 보행안전 및 편의증진에 관한 법률」 제2조제3호에 따른 보행자우선도로를 말한다.

<|user|>
질문: {question}
참고문서: {context}

<|assistant|>
**{question}**

<|answer|>
**직접답변**: {question}는 

**상세설명**: 


**법적근거**: 


**참고사항**: """,
    input_variables=["question", "context"]
)

# Few-shot 학습 강화 프롬프트
few_shot_prompt = PromptTemplate(
    template="""<|system|>
다음 예시들을 참고하여 동일한 형식과 품질로 답변하세요.

<|example1|>
질문: 자율주행시스템이란?
답변: "「자율주행자동차 상용화 촉진 및 지원에 관한 법률」 제2조 제1항 제2호에 따른 자율주행시스템을 말한다. 이 경우 그 종류는 완전 자율주행시스템, 부분 자율주행시스템 등 행정안전부령으로 정하는 바에 따라 세분할 수 있다."

<|example2|>  
질문: 보행자우선도로이란?
답변: 보행안전 및 편의증진에 관한 법률」 제2조제3호에 따른 보행자우선도로를 말한다.

<|user|>
질문: {question}
참고문서: {context}

<|assistant|>
**{question}**""",
    input_variables=["question", "context"]
)

# 3. 리트리버 설정
retriever_chroma = vectorstore.as_retriever(
    search_kwargs={
        "k": 5
        }
)
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=512,
    temperature=0.7,
)

# 1.4 LangChain 호환 래퍼 적용
llm = HuggingFacePipeline(pipeline=pipe)



Device set to use cuda:0


In [88]:
# 4. QA 체인 구성
qa_chain_chroma = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever_chroma,
    chain_type="stuff",
    chain_type_kwargs={"prompt": prompt}
)

# 3. 직접 generate() 호출
def generate_text(querry, max_length=512):
    inputs = tokenizer(querry, return_tensors="pt").to(model.device)
    outputs = model.generate(
        **inputs,
        max_length=max_length,
        temperature=0.7,
        top_p=0.9
    )
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# 실행 예시
print(generate_text("자율주행시스템"))
# # 5. 실제 질의 실행
query = "자율주행시스템에 대해 알려줘"
res_chroma = qa_chain_chroma.invoke({"query": query})

# # 6. 출력
print("✅ 필터 적용 후 >> Chroma 답변:\n", res_chroma["result"])


The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:32000 for open-end generation.
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:86: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


자율주행시스템은 차량 스스로 주행환경을 인식하여 차량의 위치와 방향, 주행 상태를 판단하고, 안전운행을 위한 제동, 조향, 가감속 등의 조치를 수행하여 안전운행을 지원한다.
자율주행시스템은 자율주행시스템의 종류에 따라 자율주행지원시스템, 자율주행시스템, 자율주행완성시스템으로 구분한다.
자율주행지원시스템은 자율주행시스템의 종류 중 자율주행시스템의 작동이 필요한 상황에서 운전자가 자율주행시스템 작동에 필요한 조작을 한 경우에만 자율주행시스템이 작동하는 시스템을 말한다.
자율주행시스템은 자율주행시스템의 종류 중 자율주행시스템의 작동이 필요한 상황에서 자율주행시스템이 작동하는 시스템을 말한다.
자율주행완성시스템은 자율주행시스템의 종류 중 자율주행시스템이 작동하는 시스템을 말한다.
자율주행시스템은 자율주행시스템의 종류에 따라 자율주행지원시스템, 자율주행시스템, 자율주행완성시스템으로 구분한다.
자율주행지원시스템은 자율주행시스템의 종류 중 자율주행시스템의 작동이 필요한 상황에서 운전자가 자율주행시스템 작동에 필요한 조작을 한 경우에만 자율주행시스템이 작동하는 시스템을 말한다.
자율주행시스템은 자율주행시스템의 종류 중 자율주행시스템의 작동이 필요한 상황에서 자율주행시스템이 작동하는 시스템을 말한다.
자율주행완성시스템은 자율주행시스템의 종류 중 자율주행시스템이 작동하는 시스템을 말한다.
자율주행시스템은 자율주행시스템의 종류에 따라 자율주행지원시스템, 자율주행시스템, 자율주행완성시스템으로 구분한다.
자율주행지원시스템은 자율주행시스템의 종류 중 자율주행시스템의 작동이 필요한 상황에서 운전자가 자율주행시스템 작동에 필요한 조작을 한


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:86: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


✅ 필터 적용 후 >> Chroma 답변:
 <|instruction|>
주어진 문서에서 질문과 가장 관련성이 높은 정보를 우선적으로 활용하여 답변하세요.
문서에 없는 내용은 추측하지 말고, 문서 기반으로만 답변하세요.

<|query|>
사용자 질문: 자율주행시스템에 대해 알려줘

<|retrieved_context|>
{"term": "자율주행시스템", "desc": ["「자율주행자동차 상용화 촉진 및 지원에 관한 법률」 제2조 제1항 제2호에 따른 자율주행시스템을 말한다.", "이 경우 그 종류는 완전 자율주행시스템, 부분 자율주행시스템 등 행정안전부령으로 정하는 바에 따라 세분할 수 있다."]}

{"term": "자율주행자동차", "desc": ["「자동차관리법」 제2조 제1호의 3에 따른 자율주행자동차로서 자율주행시스템을 갖추고 있는 자동차를 말한다."]}

{"term": "운전", "desc": ["도로(제27조제6항제3호·제44조·제45조·제54조제1항·제148조·제148조의 2 및 제156조 제10호의 경우에는 도로 외의 곳을 포함한다)에서 차마 또는 노면전차를 그 본래의 사용 방법에 따라 사용하는 것(조종 또는 자율주행시스템을 사용하는 것을 포함한다)을 말한다."]}

{"term": "자동차", "desc": ["철길이나 가설된 선을 이용하지 아니하고 원동기를 사용하여 운전되는 차(견인되는 자동차도 자동차의 일부로 본다)로서 다음 각 목의 차를 말한다.", "「자동차관리법」 제3조에 따른 다음의 자동차(승용자동차, 승합자동차, 화물자동차, 특수자동차, 이륜자동차, 「건설기계관리법」 제26조제1항 단서에 따른 건설기계). ", "다만, 원동기장치자전거는 제외한다."]}

{"term": "앞지르기", "desc": ["차의 운전자가 앞서가는 다른 차의 옆을 지나서 그 차의 앞으로 나가는 것을 말한다."]}

<|response_format|>
1. 직접 답변 (한 문장)
2. 상세 설명 (2-3문장)  
3. 법적 근거 (있는 경우)
4. 

---